# Building the App
We'll create widgets to let the user interact with the Ap. <br>

<img src="img/07_header_app.png" />

In order to run a `DataModel` instance, we'll define several inputs that the user will be customizing to request one instance of the dataset.<br>
* a `bqwidget` named `TickerAutoComplete` to select a universe
* a `Dropdown` object to select the date.
* a `Button` to trigger a data request

Note that we will store the widgets in a dictionary to facitilate their access.

In [ ]:
from bqwidgets import TickerAutoComplete
import ipywidgets

widgets = dict()

In [ ]:
# universe picker definition
widgets['universe_select'] = TickerAutoComplete(description='Select universe:', 
                                                          yellow_keys=['Index'],
                                                          style={'description_width':'initial'})

In [ ]:
# period dorpdown selector
widgets['period_select'] = ipywidgets.Dropdown(description='Period:', 
                                               options=['2010-01-01','2014-01-01'])  # for fixed income request
                                               #options=['1m','3m','6m','12m']) # for equity request

### Header box
To facilitate the display of the controls, we'll embed the widgets into a combination of boxes.

In [ ]:
# App title definition
widgets['app_title'] = ipywidgets.HTML('<h1>Heatmap</h1>')

# Button definition
button_run = ipywidgets.Button(description='Run', button_style='info')

# creation of the control box with the header
widgets['header'] = ipywidgets.VBox([widgets['app_title'],
                                     ipywidgets.HBox([widgets['universe_select'], widgets['period_select'], button_run]), 
                                    ])
widgets['header']

-----------
Now, we'll link the inputs from the user to the query sent to the `DataModel`.<br/>
Let's reformat the BQL query string based on the user inputs. 

In [ ]:
import model

In [ ]:
# set default values to widgets for bond request:
widgets['universe_select'].value = 'LEGATRUU Index'
widgets['period_select'].value = '2010-01-01'

bond_issuance = '''
    let(#amt=sum(group(amt_outstanding(currency='USD'),[year(announce_date()), month(announce_date()),cntry_of_risk()]))/1000000;) 
    get(#amt) 
    for( filter(members('{idx}'), announce_date() >= '{date}') ) 
'''.format(idx=widgets['universe_select'].value,
           date=widgets['period_select'].value)

bond_issuance

Now, the query is updated when user selects the inputs. <br>
Let's run this instance of our `DataModel` when we click on the button. 

In [ ]:
data = None
def _refresh_data(caller):
    global data
    # setup the query upon a click
    query = '''
        let(#amt=sum(group(amt_outstanding(currency='USD'),[year(announce_date()), month(announce_date()),cntry_of_risk()]))/1000000;) 
        get(#amt) 
        for( filter(members('{idx}'), announce_date() >= '{date}') ) 
    '''.format(idx=widgets['universe_select'].value,
               date=widgets['period_select'].value)
    
    # initialize the data model
    m = model.DataModel(query, asset_class='Fixed Income')
    m.run()
    
    # retrieve the data to be displayed
    data = m.build_2dim_dataset(m.data, x='Month', y='Year', v='Amount Out', calc_type='sum')
    
    # print logger
    print('Data refreshed.')
    

button_run.on_click(_refresh_data)

Scroll up and click on the blue button to trigger the refresh. <br/>
The kernel will get busy while fetching the data. 
<img src="img/04_kernel_busy.png" />

If everything goes well, you should see the following message.<br/>
<img src="img/08_data_refreshed.png" />

-----------

Now, we'll build a function that will create a heatmap object, called `GridHeatMap` from the `bqwidgets` library.

In [ ]:
from bqplot import OrdinalScale, ColorScale, GridHeatMap, Axis, Figure

def _build_heatmap(df):
    # create the scales 
    x_sc, y_sc, col_sc = OrdinalScale(), OrdinalScale(reverse=True), ColorScale(scheme='RdYlGr')
        
    # building the marks for the heatmap
    grid_map = GridHeatMap(row=df.index, column=df.columns, color=df, 
                           scales={'column': x_sc, 'row': y_sc, 'color': col_sc},
                           stroke='transparent', null_color='transparent',
                           selected_style={'opacity': 1.0}, unselected_style={'opacity': 0.4})

    # create the axis 
    ax_x, ax_y = Axis(scale=x_sc, grid_lines='none', label=df.columns.name), \
                 Axis(scale=y_sc, grid_lines='none', label=df.index.name, orientation='vertical')

    # generating the figure
    grid_ui = Figure(marks=[grid_map], axes=[ax_x, ax_y], padding_y=0.0, 
                     title='{} distribution'.format(widgets['universe_select'].value),
                     fig_margin={'bottom': 35, 'left': 150, 'right': 10, 'top': 60},
                     layout={'width':'100%', 'height':'400px'})

    # define the output object to get displayed
    return ipywidgets.VBox([grid_ui], 
                            layout={'width':'99%', 'min_height':'400px', 'overflow_x':'hidden'})


In [ ]:
_build_heatmap(data)

--------
## Generating the `app.py` file

Now we have all the necessary methods, you will create a file that store all these methods into a `Heatmap` class. <br/>

<img src="img/09_build_app_file.png" />